## Guidelines for Prompting

- We'll use two prompting principles and their related tactics in order to write effective prompts 
for large language models.

#### Setup

- Load the API key and relevant Python libaries.

**helper function**

- We will use Groq's "llama-3.3-70b-versatile" model 

In [21]:
from langchain_groq import ChatGroq
import langchain_core
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.getenv('GROQ_API_KEY')

In [30]:
# model='llama-3.3-70b-versatile'

llm = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=None
)

In [32]:
def get_completion(prompt, model="llama-3.3-70b-versatile"):
    llm.model_name = model
    return llm.invoke(prompt).content

In [48]:
print(get_completion("Tell me about Sachin Tendulkar in 100 words"))

Sachin Tendulkar is a former Indian cricketer widely regarded as one of the greatest batsmen in the history of the sport. Born on April 24, 1973, he made his international debut at 16 and went on to play for 24 years, scoring 100 international centuries and over 34,000 runs. Tendulkar won numerous awards, including the Bharat Ratna, India's highest civilian honor. He is known for his humility, dedication, and passion for cricket, inspiring millions of fans worldwide with his incredible talent and sportsmanship. He retired in 2013, leaving behind a legacy in the cricket world.


## Prompting Principles

#### Principle 1 : Write clear and specific instructions
#### Principle 2 : Give the model time to “think”

#### Tactics

**Tactic 1 : Use delimiters to clearly indicate distinct parts of the input**

- Delimiters can be anything like: ```, """, < >, <tag> </tag>, :

In [46]:
text = r"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}
"""

response = get_completion(prompt)
print(response)


To get the desired output from a model, provide instructions that are as clear and specific as possible, even if it means writing a longer prompt, to guide the model and reduce the chances of receiving irrelevant or incorrect responses.


**Tactic 2 : Ask for a structured output**

- JSON, HTML

In [57]:
prompt = r"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": 1,
    "title": "The Lost City of Eldarath",
    "author": "Ava Morales",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "Beyond the Quantum Veil",
    "author": "Ethan Kim",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Shadow of Blackwood House",
    "author": "Lila Rose",
    "genre": "Mystery"
  }
]
```


**Tactic 3 : Ask the model to check whether conditions are satisfied**

In [89]:
import warnings
warnings.filterwarnings("ignore")

text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Here are the instructions rewritten in the requested format:

Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it while the water is boiling.
Step 3 - Pour the boiling water over the tea bag once it's hot enough.
Step 4 - Let the tea sit for a bit so it can steep.
Step 5 - Take out the tea bag after a few minutes.
Step 6 - Add sugar or milk to taste, if desired.


In [77]:
text_2 = r"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = r"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
Since there is no text provided, I will respond accordingly. 

No steps provided.


**Tactic 4 : "Few-shot" prompting**

In [81]:
prompt = r"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: The tree that weathers the fiercest storm has roots that dig the deepest; the flame that burns the brightest has been tempered by the darkest night; the mountain that touches the highest sky has endured the most relentless erosion.


### Principle 2 : Give the model time to “think”

**Tactic 1 : Specify the steps required to complete a task**

In [91]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
The text tells the story of siblings Jack and Jill's unfortunate accident while fetching water from a hilltop well, but they remain undimmed in their adventurous spirits.

The French translation of the summary is: Les frères et sœurs Jack et Jill racontent l'histoire d'un accident malheureux alors qu'ils allaient chercher de l'eau à un puits au sommet d'une colline, mais leur esprit aventureux reste inchangé.

The names mentioned in the French summary are: 
1. Jack
2. Jill

```json
{
  "french_summary": "Les frères et sœurs Jack et Jill racontent l'histoire d'un accident malheureux alors qu'ils allaient chercher de l'eau à un puits au sommet d'une colline, mais leur esprit aventureux reste inchangé.",
  "num_names": 2
}
```


**Ask for output in a specified format**

In [94]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack and Jill, siblings, went on a quest to fetch water from a hilltop well, but they tripped and fell, returning home slightly battered but still adventurous.

Translation: Jack et Jill, des frères et sœurs, sont partis en quête pour chercher de l'eau à un puits au sommet d'une colline, mais ils ont trébuché et sont tombés, rentrant chez eux légèrement meurtris mais toujours aventureux.

Names: 
1. Jack
2. Jill

Output JSON: {"french_summary": "Jack et Jill, des frères et sœurs, sont partis en quête pour chercher de l'eau à un puits au sommet d'une colline, mais ils ont trébuché et sont tombés, rentrant chez eux légèrement meurtris mais toujours aventureux.", "num_names": 2}


**Tactic 2 : Instruct the model to work out its own solution before rushing to a conclusion**

In [97]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The student's solution is almost correct, but there's a small mistake in the maintenance cost.

The maintenance cost is given as a flat $100k per year, and an additional $10 / square foot. So, the maintenance cost should be $100,000 + $10x, not $100,000 + $100x.

Here's the corrected solution:

Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000

So, the total cost for the first year of operations as a function of the number of square feet is 360x + 100,000. The student's solution was incorrect due to the mistake in the maintenance cost.


- Note that the student's solution is actually not correct.

- We can fix this by instructing the model to work out its own solution first.

In [100]:
prompt = f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
```
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
To find the total cost for the first year of operations, we need to calculate the costs of land, solar panels, and maintenance.

Let x be the size of the installation in square feet.

1. Land cost: The land cost is $100 per square foot, so the total land cost is 100x.
2. Solar panel cost: The solar panel cost is $250 per square foot, so the total sola

### Model Limitations : Hallucinations

#### Boie is a real company, the product name is not real.

In [102]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

The AeroGlide UltraSlim Smart Toothbrush by Boie is a high-tech oral care device designed to provide a superior cleaning experience. Here are some key features and benefits of this toothbrush:

**Design and Build:**
The AeroGlide UltraSlim Smart Toothbrush has a sleek and slim design, making it easy to maneuver and reach all areas of the mouth. It's also lightweight and ergonomic, providing a comfortable grip.

**Brushing Technology:**
The toothbrush features advanced AeroGlide technology, which uses a combination of gentle vibrations and soft bristles to remove plaque, bacteria, and food particles from teeth. The bristles are also designed to be gentle on gums, reducing the risk of irritation and bleeding.

**Smart Features:**
The AeroGlide UltraSlim Smart Toothbrush is equipped with advanced smart features, including:

1. **Bluetooth Connectivity:** The toothbrush connects to a mobile app, allowing users to track their brushing habits, set reminders, and receive personalized feedback

**A note about the backslash**

- We are using a backslash \ to make the text fit on the screen without inserting newline '\n' characters.

- This model isn't really affected whether you insert newline characters or not. But when working with LLMs in general, 
  you may consider whether newline characters in your prompt may affect the model's performance.